<a href="https://colab.research.google.com/github/hanaluw/NLP-Topic-Modeling-for-Vietnames-Fintech-news/blob/main/Bertopic%2Bctfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers
# !pip install sentence_transformers
# !pip install pandas openpyxl  # đọc excel
!pip install -U bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

# Model

In [ ]:
import pandas as pd
df = pd.read_excel('/content/trial2018only.xlsx')
df.head()

,url,title,date,content,Fintech
0,https://cafef.vn/dat-len-ban-can-tinh-linh-hoa...,"Đặt lên bàn cân ""tính linh hoạt"" của các dòng ...",2018-06-28,"Đi cùng với sự gia tăng chất lượng cuộc sống, ...",1
1,https://cafef.vn/nhung-dieu-ban-khong-the-bo-q...,Những điều bạn không thể bỏ qua khi muốn vay t...,2018-11-19,"Với thủ tục nhanh chóng, đơn giản và gói lãi s...",1
2,https://cafef.vn/doi-sim-11-so-sang-10-so-hang...,Đổi sim 11 số sang 10 số: Hàng triệu người dùn...,2018-05-31,TIN MỚI Hàng triệu người dùng phải đến ngân hà...,1
3,https://cafef.vn/giai-phap-cong-nghe-cho-nhung...,Giải pháp công nghệ cho những khó khăn trong v...,2018-11-30,TIN MỚI Samsung Pay được cài đặt sẵn trên hầu ...,1
4,https://thesaigontimes.vn/chon-trang-web-ban-h...,Chọn trang web bán hàng giá tốt - Tạp chí Kinh...,2018-09-24,Chọn trang web bán hàng giá tốt Phúc An (SGTT)...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   url      465 non-null    object        
 1   title    465 non-null    object        
 2   date     465 non-null    datetime64[ns]
 3   content  465 non-null    object        
 4   Fintech  465 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 18.3+ KB


In [ ]:
import re
import pandas as pd

def clean1(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = re.sub(r"[“”‘’…\"']", " ", text)  # Remove special quotes
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    return text

# Apply cleaning only to valid strings
df['title'] = df['title'].astype(str).apply(clean1)
df['content'] = df['content'].astype(str).apply(clean1)

# Remove rows with empty content or title after cleaning
df = df[(df['title'].str.strip() != "") & (df['content'].str.strip() != "")]

# Convert to list for later use
title = df["title"].tolist()
content = df["content"].tolist()

# Ensure date column is datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")  # coerce invalid formats to NaT
df = df[df["date"].notna()]  # Drop rows where date conversion failed
timestamps = df["date"].tolist()

In [ ]:
import pickle
import os

# Create folder in Drive
folder_path = "/content/drive/MyDrive/Fintech_BERTopic"
os.makedirs(folder_path, exist_ok=True)

# Save all data in one file
data = {
    "title": df["title"].tolist(),
    "content": df["content"].tolist(),
    "timestamps": df["date"].tolist()
}

with open(f"{folder_path}/text_data.pkl", "wb") as f:
    pickle.dump(data, f)

##load content back

In [ ]:
import pickle

with open("/content/drive/MyDrive/Fintech_BERTopic/text_data.pkl", "rb") as f:
    data = pickle.load(f)

title = data["title"]
content = data["content"]
timestamps = data["timestamps"]

In [ ]:
from transformers import AutoTokenizer

# Tải tokenizer của mô hình
tokenizer = AutoTokenizer.from_pretrained("dangvantuan/vietnamese-document-embedding", trust_remote_code=True)

# Tìm số token của mỗi văn bản
token_lengths = [len(tokenizer.encode(text, truncation=False)) for text in content]

# Tìm chiều dài lớn nhất và chỉ số của văn bản đó
max_length = max(token_lengths)
max_index = token_lengths.index(max_length)

print(f"Chiều dài lớn nhất (số token): {max_length}")
print(f"Văn bản dài nhất ở chỉ số: {max_index}")

# Kiểm tra có bị cắt không
if max_length > 8192:
    print("\n⚠️ Văn bản này vượt quá giới hạn 8192 tokens ⇒ sẽ bị cắt khi embed.")
else:
    print("\n✅ Văn bản nằm trong giới hạn ⇒ sẽ không bị cắt.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Chiều dài lớn nhất (số token): 4117
Văn bản dài nhất ở chỉ số: 464

✅ Văn bản nằm trong giới hạn ⇒ sẽ không bị cắt.


#Pre-calculate Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True, device='cuda')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/dangvantuan/Vietnamese_impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dangvantuan/Vietnamese_impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(content, batch_size=16, show_progress_bar=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
import numpy as np

# After computing embeddings
np.save('fintechembeddings.npy', embeddings)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

folder_path = "/content/drive/MyDrive/BERTopc_Light"
os.makedirs(folder_path, exist_ok=True)

In [ ]:
np.save(f"{folder_path}/fintechembeddings.npy", embeddings)

##load embeddings back

In [ ]:
import numpy as np

# Load the embeddings
embeddings = np.load('/content/fintechembeddings.npy')

# Optional: check shape
print(embeddings.shape)

(465, 768)


# Dimensionality reduction + Clustering

In [ ]:
from bertopic import BERTopic
from umap import UMAP
umap_model = UMAP(
    n_neighbors=30,
    n_components=5,
    min_dist=0.05,
    metric='cosine',
    random_state=42,
    low_memory=True,
    n_jobs=-1
)

In [ ]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=15,            # Minimum number of docs in a topic
    metric='euclidean',             # Works well with UMAP (cosine already applied in UMAP)
    cluster_selection_method='eom', # Standard for well-separated clusters
    prediction_data=True,           # Needed if you plan to update or visualize
    core_dist_n_jobs=-1             # Use all CPU cores
)

In [ ]:
from bertopic import BERTopic

In [ ]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00


In [ ]:
from underthesea import word_tokenize

In [ ]:
vietnamese_stopwords = [
    "và", "của", "là", "có", "cho", "trong", "được", "với", "một", "những",
    "các", "đã", "đang", "đến", "này", "từ", "ra", "vào", "nếu", "cũng",
    "như", "làm", "khi", "thì", "vì", "tại", "vậy", "nhưng", "để", "cần",
    "qua", "nên", "sau", "đó", "vẫn", "nhiều", "năm", "đi", "đó", "ai",
    "bao", "bằng", "chỉ", "có thể", "giữa", "hay", "kẻ", "không", "lại",
    "lên", "lúc", "mà", "nào", "nữa", "phải", "qua", "ra", "rằng", "rất",
    "tất cả", "thế", "thấy", "theo", "thì", "trên", "trước", "tuy", "và",
    "vậy", "vì", "với", "đã", "đang", "đến", "điều", "đó", "được", "đây",
    "đó", "để", "ở", "ở đây", "ở đó", "ấy", "ấy là", "đừng", "không", "này",
    "này là", "đều", "như", "đến", "bởi", "đã", "làm", "ra", "với","người",
    "ông","bà","có","đây","bị","khi","là","của","tại","và","do","theo","với","hơn",
    "trong","về","một","những","ngoài_ra","cũng","đã","rằng","trên","đó",
    "không","chỉ","nhưng","như","các","sẽ","cùng","còn","giúp","được","nếu",
    "dù","mà","qua","bên_cạnh_đó","tuy_nhiên","song","bởi","như_vậy",
    "đồng_thời","vậy_nên","bởi_vậy","bởi_vì","thế_nên","thế_nhưng","đâu",
    "đâu_đó","tất_cả","điều_này","việc_này","này","ấy","nào","gì","vậy","thế",
    "rất","cả","mỗi","hết","bất_cứ","mọi","tuy","mỗi_khi","hễ","thật","quả_thật",
    "chính","gần","xa","hết_sức","cực_kỳ","vô_cùng","lắm","quá","bao_nhiêu",
    "nhiều","ít","nào_đó","mình","cho","việc","tin","mức","đầu","cuối","phải",
    "lên","lớn","số","ra","biết","gửi","đạt","cần","vụ","đi","tới","mang","rõ",
    "cách","phương","cuộc","cạnh","thành","đủ","gồm","tiếp_tục","sử_dụng","nhận",
    "lần","nhóm","lượng","trả","tuần","nói","vừa","%","+","-","*","/","=","<",">",
    "&","tôi",",",".","(",")",":",";","bạn"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Faster tokenizer using underthesea
def tokenize_vi(text):
    return word_tokenize(text, format="text").split()

vectorizer_model = CountVectorizer(
    tokenizer=tokenize_vi,
    stop_words=vietnamese_stopwords,
    min_df=3, #minimum document frequency threshold
    ngram_range=(1, 2),
    max_features=30000
)

In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

#BERTopic training

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,

  # Hyperparameters
  top_n_words=50, #Show top 30 most relevant words .get_topic_info()
  verbose=True
)

topics, probs = topic_model.fit_transform(content, embeddings)

2025-06-29 09:47:18,709 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-29 09:47:20,638 - BERTopic - Dimensionality - Completed ✓
2025-06-29 09:47:20,639 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-29 09:47:20,665 - BERTopic - Cluster - Completed ✓
2025-06-29 09:47:20,671 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-29 09:47:50,314 - BERTopic - Representation - Completed ✓


#LET'S GET STARTED

In [ ]:
# In ra 10 topic phổ biến nhất
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,289,0_thanh_toán_khách_hàng_ngân_hàng_bảo_hiểm,"[thanh_toán, khách_hàng, ngân_hàng, bảo_hiểm, ...",[VPB: Giá hiện tại Thay đổi Xem hồ sơ doanh ng...
1,1,79,1_ảo_tiền ảo_bitcoin_tiền,"[ảo, tiền ảo, bitcoin, tiền, đào, nhà_đầu_tư, ...",[@BTC: Giá hiện tại Thay đổi Xem hồ sơ doanh n...
2,2,52,2_vay_tiêu_dùng_lãi_suất_tín_dụng,"[vay, tiêu_dùng, lãi_suất, tín_dụng, tài_chính...",[PGS.TS. Đỗ Hoài Linh Viện Ngân hàng Tài chính...
3,3,45,3_phí_nh_atm_tăng,"[phí, nh, atm, tăng, đồng, rút_tiền, dịch_vụ, ...","[TIN MỚI Anh Thanh, giám đốc một công ty luật ..."


In [ ]:
rep_docs = topic_model.representative_docs_

# Create a map from doc text to title for quick lookup
doc_to_title = {doc_text: doc_title for doc_text, doc_title in zip(content, title)}

# Print representative docs + their titles by topic
for topic_id, docs in rep_docs.items():
    print(f"Topic {topic_id}:")
    for doc_text in docs:
        doc_title = doc_to_title.get(doc_text, "Title not found")
        print(f"  Title: {doc_title}")
    print("\n")

Topic 0:
  Title: VPBank chơi trội trong mảng ngân hàng số?
  Title: Ai cũng biết thẻ tín dụng tiện lợi, nhưng vì sao người ta còn ngần ngại sử dụng?
  Title: Fintech làm thay đổi bức tranh thị trường tài chính - Tạp chí Kinh tế Sài Gòn


Topic 1:
  Title: Lang thang với Bitcoin, mong chờ được lộ sáng
  Title: Chi 10 tỉ mua trâu cày tiền ảo, nhận được sắt vụn
  Title: Sập bẫy lừa tiền ảo


Topic 2:
  Title: Cho vay tiêu dùng Việt Nam bao giờ theo kịp thế giới?
  Title: Các nước đang quản lý cho vay ngang hàng như thế nào?
  Title: Quản lý cho vay ngang hàng: Kinh nghiệm quốc tế và một số gợi ý cho Việt Nam


Topic 3:
  Title: Phí ngân hàng tăng, doanh nghiệp bức xúc
  Title: Vì sao ngân hàng tăng phí dịch vụ?
  Title: Choáng với rừng phí dịch vụ ngân hàng




In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def get_ctfidf_similarity_scores(topic_model, docs, topics):
    # Bước 1: Lấy vectorizer và mô hình c-TF-IDF
    vectorizer = topic_model.vectorizer_model
    ctfidf_model = topic_model.ctfidf_model

    # Bước 2: Transform văn bản thành BoW rồi c-TF-IDF vector
    X_bow = vectorizer.transform(docs)
    X_ctfidf = ctfidf_model.transform(X_bow)

    # Bước 3: Lấy ma trận c-TF-IDF của các topic
    topic_ctfidf = topic_model.c_tf_idf_
    topic_info = topic_model.get_topic_info()
    topic_id_to_index = {tid: i for i, tid in enumerate(topic_info["Topic"].tolist())}

    # Bước 4: Tính similarity giữa mỗi văn bản với topic được gán
    similarity_scores = []
    for i in range(len(docs)):
        topic_id = topics[i]
        if topic_id == -1 or topic_id not in topic_id_to_index:
            similarity_scores.append(None)
        else:
            topic_index = topic_id_to_index[topic_id]
            doc_vec = X_ctfidf[i]
            topic_vec = topic_ctfidf[topic_index]
            sim = cosine_similarity(doc_vec, topic_vec)[0][0]
            similarity_scores.append(sim)

    return similarity_scores

In [ ]:
similarity_scores = get_ctfidf_similarity_scores(topic_model, content, topics)

df_similarity = pd.DataFrame({
    "Title": title,
    "Assigned Topic": topics,
    "Similarity to Topic (c-TF-IDF)": similarity_scores
})


In [ ]:
df_similarity

,Title,Assigned Topic,Similarity to Topic (c-TF-IDF)
0,Đặt lên bàn cân tính linh hoạt của các dòng sả...,0,0.138201
1,Những điều bạn không thể bỏ qua khi muốn vay t...,2,0.185872
2,Đổi sim 11 số sang 10 số: Hàng triệu người dùn...,3,0.148015
3,Giải pháp công nghệ cho những khó khăn trong v...,0,0.081031
4,Chọn trang web bán hàng giá tốt - Tạp chí Kinh...,0,0.170042
...,...,...,...
460,"Lang thang với Bitcoin, mong chờ được lộ sáng",1,0.537492
461,VPBank chơi trội trong mảng ngân hàng số?,0,0.489531
462,"Ai cũng biết thẻ tín dụng tiện lợi, nhưng vì s...",0,0.489895
463,Cho vay tiêu dùng Việt Nam bao giờ theo kịp th...,2,0.531828


In [ ]:
df_similarity.to_excel("similarity_scores.xlsx", index=False, engine="openpyxl")

# Load for visual

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

folder_path = "/content/drive/MyDrive/BERTopic_Light"
print(os.listdir(folder_path))

['topic_embeddings.safetensors', 'topics.json', 'config.json', 'ctfidf.safetensors', 'ctfidf_config.json']


In [ ]:
from bertopic import BERTopic

In [ ]:
loaded_model = BERTopic.load("/content/drive/MyDrive/BERTopic_Light")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

2025-06-29 14:57:00,188 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


In [ ]:
loaded_model.visualize_hierarchy()

2025-06-29 14:57:07,191 - BERTopic - WARNING: No c-TF-IDF matrix was found despite it is supposed to be used (`use_ctfidf` is True). Defaulting to semantic embeddings.


In [ ]:
fig1 =loaded_model.visualize_hierarchy()
fig1.write_html("/content/hierarchy_final.html")
fig1

2025-06-29 14:57:21,262 - BERTopic - WARNING: No c-TF-IDF matrix was found despite it is supposed to be used (`use_ctfidf` is True). Defaulting to semantic embeddings.
